In [16]:
import tkinter
from tkinter import *
from tkinter import filedialog 
import pandas as pd
import numpy as np

import time

def edit_distance(s1, s2):
    m = len(s1)
    n = len(s2)

    # initialize the matrix
    d = [[0] * (n + 1) for i in range(m + 1)]

    # fill the first row and first column
    for i in range(1, m + 1):
        d[i][0] = i
    for j in range(1, n + 1):
        d[0][j] = j

    # fill the rest of the matrix
    for j in range(1, n + 1):
        for i in range(1, m + 1):
            if s1[i - 1] == s2[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                d[i][j] = min(d[i - 1][j], d[i][j - 1], d[i - 1][j - 1]) + 1

    # return the final value in the matrix
    return d[m][n]

 


def count_duplicates(records):
    count_dict = {}

    for row in records:
        record = tuple(row[1:]) # discard column 1

        if record in count_dict:
            count_dict[record] ['count']+= 1
            count_dict[record]['ids'].append(row[0]) # add ID to list of IDs

        else:
            count_dict[record] = {'count': 1, 'ids': [row[0]]}

    duplicates = []
    total=0         
    for record, count_dict in count_dict.items():
         if count_dict['count'] > 1:
            duplicates.append(f"IDs: {', '.join(str(id) for id in count_dict['ids'])},{record}: duplicate count: {count_dict['count']} ")
            total=total+count_dict['count']
            
    lbl_total=Label(window, text="Number of Duplicates: {}".format(total))
    lbl_total.pack()            
    
    return duplicates

start_time = time.time()



def upload_file():
    
    file=filedialog.askopenfilename(title="Select file" , filetypes=[("CSV files", '.csv')])
    if(file):
        lbl_my_file=Label(window, text="File read!", font=("arial", 12, "bold") )
        lbl_my_file.pack()
        
        comparison=0
        threshold=5
        chunk_size=50
        
        probable=[]
        probable_dup=[]
        
        df = pd.read_csv(file, infer_datetime_format=True, encoding='ISO-8859-1')
        
        file_name.set(file)
        
        # Clean up the data
        # df.dropna(subset=['name', 'age', 'age_unit', 'province_id', 'district_id', 'municipality_id', 'tole', 'ward', 'caste', 'sex'], inplace=True)
        df.dropna(subset=['name', 'age', 'gender', 'number'], inplace=True)
        
        df["name"] = df["name"].str.replace(" ", "")
        # std = df[['id', 'name', 'age', 'age_unit', 'province_id', 'district_id', 'municipality_id',
        #           'tole', 'ward', 'caste', 'sex']].values
        std = df[['id', 'name', 'age', 'gender', 'number']].values
        
        # df['std_att'] = df.apply(lambda row: str(row['name']) + str(row['age']) + str(row['age_unit'])
        #                          + str(row['province_id']) + str(row['district_id']) + str(row['municipality_id']) 
        #                          + str(row['tole']) + str(row['ward']) + str(row['caste']) + str(row['sex']), axis=1)
        df['std_att'] = df.apply(lambda row: str(row['name']) + str(row['age']) + str(row['gender'])
                                 + str(row['number']), axis=1)
        # std_att = df['name'].astype(str) + df['age'].astype(str) + df['age_unit'].astype(str) 
        std_att = df['name'].astype(str) + df['age'].astype(str) + df['gender'].astype(str) + df['number'].astype(str) 
    
        print("............Dataset..........")
        for index, row in df.iterrows():
            a=f"ID: {row['id']}, std_att: {row['std_att']}"
            print(a)
    
    
        mylist = Text(window, height=15, width=150)
        mylist.pack(side=TOP, padx=10, pady=10, expand=True)

              
        mylist1 = Text(window, height=15, width=150)
        mylist1.pack(side=BOTTOM, padx=10, pady=10, expand=True)
        
        unique_ids = [] # list to store unique IDs
        unique_std_att = [] # list to store corresponding unique std_att values
       
       
        
        print("...........Edit Distance without sorting: ...........")
        for i in range(0, len(df), chunk_size):
            chunk = df.iloc[i:i+chunk_size]
            std_chunk=std_att.iloc[i:i+chunk_size]
            m = len(chunk)
            for j in range(m):
                for k in range(j + 1, m):
                    id_j = str(chunk.iloc[j]['id'])
                    id_k = str(chunk.iloc[k]['id'])
                    dist = edit_distance(std_chunk.iloc[j], std_chunk.iloc[k])
                    comparison += 1 # m * (m - 1) / 2
                    if dist == 2 or dist ==1 :
                        edit = (dist, f"Edit distance between ({id_j}){chunk.iloc[j]['std_att']} and ({id_k}){chunk.iloc[k]['std_att']} : {dist}")
                        probable_dup.append(edit)
                        
                      
                        
                    if dist <= threshold:                       
                        pair = (dist, f"Edit distance between ({id_j}){chunk.iloc[j]['std_att']} and ({id_k}){chunk.iloc[k]['std_att']} : {dist}")
                        probable.append(pair)    
                        
                        if id_j not in unique_ids:
                            unique_ids.append(id_j)
                            unique_std_att.append((chunk.iloc[j]['id'], chunk.iloc[j]['std_att']))
                            
                        if id_k not in unique_ids:
                            unique_ids.append(id_k)
                            unique_std_att.append((chunk.iloc[k]['id'], chunk.iloc[k]['std_att']))
                            
        print("...........Edit Distance in Sorted order.............")                    
        probable = sorted(probable, key=lambda x: x[0]) # sort by distance
        for pair in probable:
            print(pair[1])   
        
        probable_dup = sorted(probable_dup, key=lambda x: x[0]) # sort by distance
        mylist1.insert(END,f"Probable Dupliates:\n")
        z = 1
        for edit in probable_dup:
            mylist1.insert(END,f"{z}: {edit[1]}\n")
            z=z+1
        mylist1.config(state='disabled')

              
        print("..........Dataset within the edit distance with unique id.............")
        print(unique_std_att)
        print("Length of unique attribute(edit distance): ", len(unique_std_att))

        duplicates = count_duplicates(unique_std_att)
        mylist.insert(END,f"Exact Dupliates:\n")
        for i in range(len(duplicates)):
            mylist.insert(END,f"{i+1}: {duplicates[i]}\n" )
        mylist.config(state='disabled')
        
        lbl_data=Label(window, text="Number of data in dataset: {}".format(len(std_att)))
        lbl_data.pack()
             
        lbl_comparison=Label(window, text="Number of Comparison: {}".format(comparison))
        lbl_comparison.pack()            
                

window=Tk()
window.geometry("1000x800")
window.title("Data Redundancy Detection System")
title_lbl=Label(window, text="Upload .csv file", font=("arial", 30, "italic bold"),bd=7)
title_lbl.pack()
button=Button(window, text="Upload File", relief=RAISED,font=("arial", 15, "bold"), width=15, bg="light blue" ,command=upload_file)
button.pack()

file_name=StringVar()
file_data=StringVar()
my_file=StringVar()

lbl_name=Label(window, textvariable=file_name)
lbl_name.pack()


window.mainloop()




end_time = time.time()
processing_time = end_time - start_time
print(f"\n \n \n Processing time: {processing_time} seconds")

C:\Users\Bipin Shrestha\AppData\Local\Temp\ipykernel_18192\2895056622.py:79: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(file, infer_datetime_format=True, encoding='ISO-8859-1')


............Dataset..........
ID: 1, std_att: bipin22male9800000000
ID: 2, std_att: bishal20female9811111111
ID: 3, std_att: ajay22male9866844509
ID: 4, std_att: bipin22male9800000000
ID: 5, std_att: rita19female9812345678
ID: 6, std_att: bipin22male9800000088
ID: 7, std_att: ram80male9811111111
ID: 8, std_att: sit22female9800000000
ID: 9, std_att: oracle22female9196856201
ID: 10, std_att: oracle90female9196856201
...........Edit Distance without sorting: ...........
...........Edit Distance in Sorted order.............
Edit distance between (1)bipin22male9800000000 and (4)bipin22male9800000000 : 0
Edit distance between (1)bipin22male9800000000 and (6)bipin22male9800000088 : 2
Edit distance between (4)bipin22male9800000000 and (6)bipin22male9800000088 : 2
Edit distance between (9)oracle22female9196856201 and (10)oracle90female9196856201 : 2
..........Dataset within the edit distance with unique id.............
[(np.int64(1), 'bipin22male9800000000'), (np.int64(4), 'bipin22male980000000